# Upload faiss indices to HuggingFace

# Embedding testing

In [8]:
import json
import glob
import itertools

import os
import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModel, CLIPProcessor, CLIPModel
from pytubefix import YouTube
import cv2
from youtube_transcript_api import YouTubeTranscriptApi
import re
from typing import List, Dict, Tuple, Optional
import pandas as pd
from tqdm import tqdm


In [13]:
from pytubefix import Channel

c = Channel('https://www.youtube.com/@3blue1brown/videos')
print(f'Channel name: {c.channel_name}')

# Loop through each video in the channel
for video in c.videos:
    if video and video.streams:
        # Try getting the lowest resolution first
        stream = video.streams.get_lowest_resolution()
        
        # If lowest resolution is not available, try default resolution (e.g., 360p)
        if stream is None:
            stream = video.streams.filter(res="360p").first()
        
        # If 360p is not available, get the highest available resolution
        if stream is None:
            stream = video.streams.get_highest_resolution()

        if stream:
            video_id = video.video_id  # Get the YouTube video ID
            filename = f"{video_id}.mp4"  # Save as {video_id}.mp4
            
            # Download the video
            stream.download(output_path="videos", filename=filename)
            print(f"Downloaded: {filename} ({stream.resolution})")
        else:
            print(f"Skipping video {video.video_id}: No valid stream found")
    else:
        print(f"Skipping video {video}: No streams available")

Channel name: 3Blue1Brown
Downloaded: 6dTyOl1fmDo.mp4 (360p)
Downloaded: hFMaT9oRbs4.mp4 (360p)
Downloaded: YdOXS_9_P4U.mp4 (360p)
Downloaded: IQqtsm-bBRU.mp4 (360p)
Downloaded: LPZh9BOjkQs.mp4 (360p)
Downloaded: piJkuavhV50.mp4 (360p)
Downloaded: rbu7Zu5X1zI.mp4 (360p)
Downloaded: EmKQsSDlaa4.mp4 (360p)
Downloaded: 9-Jl0dxWQs8.mp4 (360p)
Downloaded: W3I3kAg2J7w.mp4 (360p)
Downloaded: eMlx5fFNoYc.mp4 (360p)
Downloaded: wjZofJX0v4M.mp4 (360p)
Downloaded: Cz4Q4QOuoo8.mp4 (360p)
Downloaded: KTzGBJPuJwM.mp4 (360p)
Downloaded: 6a1fLEToyvU.mp4 (360p)
Downloaded: aXRTczANuIs.mp4 (360p)
Downloaded: QCX62YJCmGk.mp4 (360p)
Downloaded: d_qvLDhkg00.mp4 (360p)
Downloaded: YtkIWDE36qU.mp4 (360p)
Downloaded: NOCsdhzo6Jg.mp4 (360p)
Downloaded: IaSGqQa5O-M.mp4 (360p)
Downloaded: cy8r7WSuT1I.mp4 (360p)
Downloaded: zeJD6dqJ5lo.mp4 (360p)
Downloaded: KuXjwB4LzSA.mp4 (360p)
Downloaded: 851U557j6HE.mp4 (360p)
Downloaded: cDofhN-RJqg.mp4 (360p)
Downloaded: VYQVlVoWoPY.mp4 (360p)
Downloaded: bOXCLR3Wric.mp4 (

In [66]:
print(os.getcwd())

C:\Users\panup\Documents\Tampereen Yliopisto\COMP.CS.530 Fine-tuning Large Language Models\capstone


In [69]:
from embeddings import MultimodalEmbedder

In [70]:
ARXIV_METADATA_FILEPATH = glob.glob("tmp/**/arxiv*", recursive=True)[0]
print(f"{ARXIV_METADATA_FILEPATH = }")

ARXIV_METADATA_FILEPATH = 'tmp\\arxiv-metadata-oai-snapshot.json'


In [71]:
# metadata = {}
# transcript = YouTubeTranscriptApi.get_transcript("eMlx5fFNoYc")
# transcript_text = " ".join([item['text'] for item in transcript])
# metadata["transcript"] = transcript_text

In [72]:
# metadata["transcript"]

In [73]:
embedder = MultimodalEmbedder()

# Example text documents (e.g., from arXiv)
documents = [
    {
        "id": "2104.08663",
        "title": "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks",
        "source": "arXiv",
        "text": "Large pre-trained language models have been shown to store factual knowledge in their parameters, and achieve state-of-the-art results when fine-tuned on downstream NLP tasks. However, their ability to access and precisely manipulate knowledge is still limited, and hence on knowledge-intensive tasks, their performance lags behind task-specific architectures. Additionally, providing provenance for their decisions and updating their world knowledge remains challenging. Pre-trained models with a differentiable access mechanism to explicit nonparametric memory can overcome this limitation, but have so far been only investigated for extractive downstream tasks. We explore a general-purpose fine-tuning recipe for retrieval-augmented generation (RAG) — models which combine pre-trained parametric and non-parametric memory for language generation. We introduce RAG models where the parametric memory is a pre-trained seq2seq model and the non-parametric memory is a dense vector index of Wikipedia, accessed with a pre-trained neural retriever. We compare two RAG formulations, one which conditions on the same retrieved passages across the entire generated text, and another which can use different passages per token. We fine-tune and evaluate our models on a wide range of knowledge-intensive NLP tasks and set the state-of-the-art on three open domain QA tasks, outperforming parametric seq2seq models and task-specific retrieve-and-extract architectures. For language generation tasks, we find that RAG models generate more specific, diverse and factual language than a state-of-the-art parametric-only seq2seq baseline."
    },
    {
        "id": "2103.00020",
        "title": "CLIP: Connecting Text and Images",
        "source": "arXiv",
        "text": "We present a neural network that efficiently learns visual concepts from natural language supervision. Our method can be applied to any visual concept that people can describe in language and is trained using pairs of images and text found across the internet. By design, the network can be instructed in natural language to perform a wide variety of classification benchmarks, without directly optimizing for the benchmark's performance, similar to the \"zero-shot\" capabilities of GPT-2 and 3. We find that this approach is efficient and scalable, achieving good performance on a variety of image classification datasets. It also enables flexible zero-shot transfer, where a single model can be adapted to perform many different tasks."
    }
]

# Example images (paths would be different in your system)
images = [
    # {
    #     "id": 1,
    #     "path": "tmp/image1.png",
    #     "caption": "Diagram of a Retrieval Augmented Generation system",
    #     "source": "Research paper figure"
    # },
]

# Example YouTube videos related to academic research
youtube_videos = [
    "eMlx5fFNoYc",  # Attention in transformers, step-by-step | DL6
]

# Index content
embedder.index_documents(documents)
embedder.index_images(images)
embedder.index_youtube_videos(youtube_videos)
embedder.save_indices(os.path.join("tmp", "faiss_indices"))

# Example search
print("Example search:")
print("To run a search, load the indices first or index some content")
print("results = embedder.search('multimodal embeddings for scientific papers', k=3)")

# Example of loading saved indices
# embedder.load_indices()
# results = embedder.search('multimodal embeddings for scientific papers', k=3)
# print(results)

Loading text embedding model...
Loading image/video embedding model...


Indexing videos:   0%|                                                                           | 0/1 [00:00<?, ?it/s]

tmp\videos\eMlx5fFNoYc.mp4


Indexing videos: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.40s/it]

Example search:
To run a search, load the indices first or index some content
results = embedder.search('multimodal embeddings for scientific papers', k=3)


In [78]:
#embedder.load_indices(os.path.join("tmp", "faiss_indices")))
results = embedder.search('CLIP: Connecting Text and Images', k=3)
print(results.keys())
print(*results["video"], sep="\n")

dict_keys(['text', 'image', 'video'])
{'video_id': 'eMlx5fFNoYc', 'title': 'Attention in transformers, step-by-step | DL6', 'description': 'Demystifying attention, the key mechanism inside transformers and LLMs.\nInstead of sponsored ad reads, these lessons are funded directly by viewers: https://3b1b.co/support\nSpecial thanks to these supporters: https://www.3blue1brown.com/lessons/attention#thanks\nAn equally valuable form of support is to simply share the videos.\n\nDemystifying self-attention, multiple heads, and cross-attention.\nInstead of sponsored ad reads, these lessons are funded directly by viewers: https://3b1b.co/support\n\nThe first pass for the translated subtitles here is machine-generated, and therefore notably imperfect. To contribute edits or fixes, visit https://www.criblate.com\n\nЗвуковая дорожка на русском языке: Влад Бурмистров.\n\nAnd yes, at 22:00 (and elsewhere), "breaks" is a typo.\n\n------------------\n\nHere are a few other relevant resources\n\nBuild a 

In [3]:
from pytubefix import YouTube
from pytubefix.cli import on_progress

url = "https://www.youtube.com/watch?v=eMlx5fFNoYc"

yt = YouTube(url, on_progress_callback=on_progress)
print(yt.title)

ys = yt.streams.get_highest_resolution()
ys.download()

Attention in transformers, step-by-step | DL6


'C:\\Users\\panup\\Documents\\Tampereen Yliopisto\\COMP.CS.530 Fine-tuning Large Language Models\\capstone\\Attention in transformers, step-by-step  DL6.mp4'